In [147]:
import numpy as np
import pandas as pd
from logistic_dml import DML
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier,RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import LinearSVR,LinearSVC
from sklearn.linear_model import LogisticRegression, LinearRegression
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold, f_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer, PowerTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow import keras 
from scipy import stats
from sklearn.linear_model import SGDClassifier, SGDRegressor


In [96]:
df = pd.read_csv( '../../Dataset2_BirthAge20-34_subset10000.csv', low_memory=False)

In [207]:
df[df.birth==0].shape

(5947, 23)

In [98]:
train =df.drop(['Unnamed: 0'],axis=1)

In [168]:
Y = train['birth']
A = train['pill']
X = train.drop(['pill','birth'], axis=1)

In [190]:
# XA_rs, y_rs = RandomUnderSampler(sampling_strategy='majority',
#                                  random_state=0,
#                                  replacement=False).fit_resample(pd.concat((A, X), axis=1), train.birth)
# # XA_rs = XA_rs.loc[:, XA_rs.nunique() > 1]
# X_rs, A_rs = XA_rs.drop('pill', axis=1), XA_rs['pill']
K=5

In [191]:
A_rs[A_rs==0]


207     0
2239    0
924     0
6985    0
4789    0
       ..
9988    0
9989    0
9990    0
9991    0
9998    0
Name: pill, Length: 6113, dtype: int64

In [213]:
gbc = GradientBoostingClassifier()
gbr=GradientBoostingRegressor()
# gbr_param={'n_estimators':[10,100,500,1000]}


result_gb = DML(
        Y=Y,
        A=A,
        X=X,classifier=gbc, regressor=gbr,random_seed=0, k_folds=K,
    # M_hat_classifier=GradientBoostingClassifier(),
    #          a_hat_classifier=GradientBoostingClassifier()
)      


result_gb.train()

lb_gb, ub_gb, mean_gb, sd_gb = result_gb.significance_testing()

print('beta hat', result_gb.beta_hat)
print('p value', result_gb.p_value)
print('ci', result_gb.ci)
print('lb', lb_gb)
print('ub', ub_gb)


Getting predictions for fold k = 1
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 2
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 3
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 4
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 5
Predicting M_hat
Predicting a_hat
Predicting t_hat
Getting predictions for fold k = 2
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 2
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 3
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 4
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 5
Predicting M_hat
Predicting a_hat
Predicting t_hat
Getting predictions for fold k = 3
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting prediction

/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, ax

In [193]:
\```/ 

SyntaxError: unexpected character after line continuation character (1768028776.py, line 1)

In [211]:
rfr = RandomForestRegressor()
rfc= RandomForestClassifier()
# rf_param={'n_estimators': (10, 30, 50, 100, 200),'max_features': (1,2,3)
# }|
result_rf = DML(
        Y=Y,
        A=A,
        X=X,classifier=rfc, regressor=rfr,
    random_seed=0,
    k_folds=K,    

)      

                
result_rf.train()

lb_rf, ub_rf, mean_rf, sd_rf = result_rf.significance_testing()
print('beta hat', result_rf.beta_hat)
print('p value', result_rf.p_value)
print('ci', result_rf.ci)
print('lb', lb_rf)
print('ub', ub_rf)


Getting predictions for fold k = 1
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 2
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 3
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 4
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 5
Predicting M_hat
Predicting a_hat
Predicting t_hat
Getting predictions for fold k = 2
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 2
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 3
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 4
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 5
Predicting M_hat
Predicting a_hat
Predicting t_hat
Getting predictions for fold k = 3
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting prediction

/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, ax

In [210]:
svr = SVR(kernel='linear')
svc= SVC(probability=True,kernel='linear')
result_sv = DML(
        Y=Y,
        A=A,
        X=X,classifier=svc, regressor=svr,random_seed=0, k_folds=K,
        # M_hat_classifier=SVC(probability=True,kernel='linear'),
        # a_hat_classifier=SVC(probability=True,kernel='linear'),)      
)

result_sv.train()

lb_sv, ub_sv, mean_sv, sd_sv = result_sv.significance_testing()
print('beta hat', result_sv.beta_hat)
print('p value', result_sv.p_value)
print('ci', result_sv.ci)
print('lb', lb_sv)
print('ub', ub_sv)

Getting predictions for fold k = 1
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 2
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 3
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 4
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 5
Predicting M_hat
Predicting a_hat
Predicting t_hat
Getting predictions for fold k = 2
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 2
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 3
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 4
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 5
Predicting M_hat
Predicting a_hat
Predicting t_hat
Getting predictions for fold k = 3
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting prediction

/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, ax

In [212]:


model_nnr = keras.Sequential([ 
    keras.layers.Dense(5, input_shape=(X_rs.shape[1],), 
                       activation='sigmoid'),keras.layers.Dense(5, activation='relu'),keras.layers.Dense(1)]) 
model_nnr.compile( 
    optimizer='adam', 
    loss='mean_squared_error', 
    metrics=['accuracy']) 
model_nnc = keras.Sequential([ 
    keras.layers.Dense(5, input_shape=(X_rs.shape[1],), 
                       activation='relu'),keras.layers.Dense(5, activation='relu'),keras.layers.Dense(units=1, activation='sigmoid')
]) 
model_nnc.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 
result_nn = DML(
        Y=y_rs,
        A=A_rs,
        X=X_rs,classifier=model_nnc , regressor=model_nnr,random_seed=0, k_folds=K)      


result_nn.train()

lb_nn, ub_nn, mean_nn, sd_nn = result_sv.significance_testing()
print('beta hat', result_nn.beta_hat)
print('p value', result_nn.p_value)
print('ci', result_nn.ci)
print('lb', lb_nn)
print('ub', ub_nn)

/Users/cheapp-pc01-04/anaconda3/envs/parttime/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Getting predictions for fold k = 1
Predicting M_hat
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 312us/step - accuracy: 0.5317 - loss: 0.7502  


AttributeError: 'Sequential' object has no attribute 'predict_proba'

for tensorflow<=2.5, predict_proba is  deprectaed and no longer in use

In [217]:
data = {'ML':['GB','RF','SVM'],'Beta':[result_gb.beta_hat,result_rf.beta_hat,result_sv.beta_hat],
       'CI_LB':[result_gb.ci[0],result_rf.ci[0],result_sv.ci[0]], 
        "CI_UB":[result_gb.ci[1],result_rf.ci[1],result_sv.ci[1]],
        "p_val":[result_gb.p_value,result_rf.p_value,result_sv.p_value]}
                        

In [219]:
data=pd.DataFrame(data)
data

,ML,Beta,CI_LB,CI_UB,p_val
0,GB,-0.164960,-0.164960,-0.164960,0.0
1,RF,-1.874448,-1.874448,-1.874448,0.0
2,SVM,-0.001983,-0.001983,-0.001983,0.0
